# Jet Tagging with Graph Neural Networks (GNN)

This notebook shows how to use a GNN for jet classification between QCD and TT jets using particle-level features represented as a graph. Note that you get some GPU time on Kaggle for free (30 hrs/week). This will make training the model here faster, but note that it won't help you with the graph building, which also does take time here. Sadly you can't turn it on just to run the training, the notebook session must be run as either CPU or GPU. If you want to experiment with GNNs and the graph building becomes annoying, just save the graphs, and then load them as you need. 

## What is a GNN?
A Graph Neural Network (GNN) is a type of neural network designed to process graph-structured data. It can learn from both node features and the graph structure. It can be used to predict information about nodes, links between nodes or graphs as a whole. 

## Why use GNNs for Jet Tagging?
- Very flexible, can start from images or dataframes, or use both 
- Has the potential to remove areas where there is no energy deposited
- Has the potential to include long range interactions in a way a CNN can't 
- Can handle variable number of particles

In [23]:
!pip install torch_geometric

## 1. Load and Prepare Data

We will create the graphs from images, but there are many other ways to do it. 

The energy deposited in a pixel will be the node feature, and we will connect each pixel to its eight closest pixels (up, down, sides, diagonals). 

The graph building function written here has an option to set max_nodes, which will only choose the top n most energetic nodes. There's also an option whether to consider all nodes or not, if you set this to false, it will throw away all the pixels with 0 energy deposited. 

In [24]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data, DataLoader
from utils import load_images
from gnn_utils import create_graph_data
from plotting_utils import plot_confusion_matrix, plot_training_history, plot_roc_curve


In [25]:
# Load data
X_train, y_train, train_ids, X_val, y_val, val_ids, X_test, test_ids = load_images()

In [26]:
# Convert to graph format - this might take a bit of time to run 
X_train_graphs = create_graph_data(X_train, y_train, max_nodes=500, consider_all_nodes=True)
X_val_graphs = create_graph_data(X_val, y_val, max_nodes=500)
X_test_graphs = create_graph_data(X_test, max_nodes=500)


Created 3520 graphs
Average number of nodes: 500.0
Average number of edges: 4226.9
Created 502 graphs
Average number of nodes: 500.0
Average number of edges: 4227.2
Created 1005 graphs
Average number of nodes: 500.0
Average number of edges: 4226.7


## 2. Build GNN Model

We'll create a GNN with:
- Graph Convolutional layers to learn from node features and graph structure
- Global pooling to get graph-level representations
- Dense layers for classification

In [27]:
class GNN(nn.Module):
    def __init__(self, num_features):
        super(GNN, self).__init__()
        # Graph convolution layers
        self.conv1 = GCNConv(num_features, 64)
        self.conv2 = GCNConv(64, 32)
        
        # Dense layers
        self.fc1 = nn.Linear(32, 16)
        self.fc2 = nn.Linear(16, 1)
        
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        
        # Graph convolution layers
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.2, training=self.training)
        x = F.relu(self.conv2(x, edge_index))
        
        # Global pooling
        x = global_mean_pool(x, batch)
        
        # Dense layers
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        
        return torch.sigmoid(x)

# Create model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GNN(num_features=1).to(device)  # 4 features: pt, eta, phi, charge
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [28]:
train_loader = DataLoader(X_train_graphs, batch_size=32, shuffle=True)
val_loader = DataLoader(X_val_graphs, batch_size=32, shuffle=True)
test_loader = DataLoader(X_test_graphs, batch_size=32)

/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


## 3. Train Model

We'll train the GNN with:
- Adam optimizer
- Binary Cross Entropy loss
- Early stopping based on validation accuracy


You'll see this code differs a fair bit from the previous DNN and CNN. This is because we're using pytorch geometric -  the most common GNN package. It integrates with Pytorch, so we are using Pytorch instead of keras. Torch is growing to be the most popular ML libary, but keras is easier to learn. 

You might also see that training on CPU this is a lot slower than the other ML methods. Can you explain why? 

In [29]:
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        optimizer.zero_grad()
        out = model(data.to(device))
        loss = criterion(out, data.y.view(-1, 1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(train_loader.dataset)

def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        out = model(data.to(device))
        loss = criterion(out, data.y.view(-1, 1))
        pred = (out > 0.5).float()
        correct += int((pred == data.y.view(-1, 1)).sum())
    return correct / len(loader.dataset), loss


history = {
    'loss': [],
    'val_loss': [],
    'accuracy': [],
    'val_accuracy': []
}


# Training loop
best_acc = 0
for epoch in range(500):
    loss = train()
    train_acc = test(train_loader)[0]
    val_acc, val_loss = test(val_loader)
    print(f'Epoch {epoch:03d}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}')
    
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), 'best_model.pt')


    # Append to history
    history['loss'].append(loss)
    history['val_loss'].append(val_loss)
    history['accuracy'].append(train_acc)
    history['val_accuracy'].append(val_acc)

Epoch 000, Loss: 0.6183, Train Acc: 0.6557, Val Acc: 0.6633
Epoch 001, Loss: 0.5809, Train Acc: 0.6557, Val Acc: 0.6633
Epoch 002, Loss: 0.5534, Train Acc: 0.7173, Val Acc: 0.7231
Epoch 003, Loss: 0.5482, Train Acc: 0.7645, Val Acc: 0.7789
Epoch 004, Loss: 0.4995, Train Acc: 0.8116, Val Acc: 0.8147
Epoch 005, Loss: 0.4630, Train Acc: 0.8321, Val Acc: 0.8367
Epoch 006, Loss: 0.4461, Train Acc: 0.8432, Val Acc: 0.8426
Epoch 007, Loss: 0.4326, Train Acc: 0.8455, Val Acc: 0.8486
Epoch 008, Loss: 0.4086, Train Acc: 0.8298, Val Acc: 0.8207
Epoch 009, Loss: 0.4227, Train Acc: 0.8321, Val Acc: 0.8247
Epoch 010, Loss: 0.4340, Train Acc: 0.8497, Val Acc: 0.8566
Epoch 011, Loss: 0.4184, Train Acc: 0.8327, Val Acc: 0.8187
Epoch 012, Loss: 0.4008, Train Acc: 0.8429, Val Acc: 0.8506
Epoch 013, Loss: 0.4157, Train Acc: 0.8560, Val Acc: 0.8526
Epoch 014, Loss: 0.4292, Train Acc: 0.8509, Val Acc: 0.8566
Epoch 015, Loss: 0.4128, Train Acc: 0.8517, Val Acc: 0.8466
Epoch 016, Loss: 0.4126, Train Acc: 0.85

OutOfMemoryError: CUDA out of memory. Tried to allocate 38.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 20.12 MiB is free. Process 3337 has 14.72 GiB memory in use. Of the allocated memory 14.12 GiB is allocated by PyTorch, and 475.08 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

## 4. Evaluate Model

Let's evaluate our model's performance on the test set.

In [ ]:
# Load best model
model.load_state_dict(torch.load('best_model.pt'))

# Evaluate on test set
model.eval()
y_true = []
y_pred = []

with torch.no_grad(): 
    for data in val_loader:
        out = model(data.to(device))
        y_true.extend(data.to('cpu').y.numpy())
        y_pred.extend(out.to('cpu').numpy())

y_pred = np.array([x[0] for x in y_pred])
pred_discrete = np.where(y_pred > 0.5, 1, 0)
# Plot confusion matrix
plot_confusion_matrix(y_true, pred_discrete)

In [ ]:
plot_roc_curve(y_true, y_pred)

# 5 - Making predictions

In [ ]:
y_pred_test = []
for data in test_loader:
    with torch.no_grad():
        output = model(data.to(device))
        # could you change the prediction threshold? Would that make it better?
        y_pred_test.extend(output.to('cpu').numpy())

y_pred_test = np.array([x[0] for x in y_pred_test])


In [ ]:
import pandas as pd
solution = pd.DataFrame({'id':test_ids, 'label':y_pred_test})
solution.to_csv('solution.csv', index=False)